In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Postal Code extraction

In [50]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
get_text = requests.get(link).text
xml = BeautifulSoup(get_text, 'xml')
table=xml.find('table')
columns = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = columns)

# Search for postcodes, borough and neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [51]:
df.head(12)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


# Data treatment

In [52]:
#1)Bring just rows where Borough is diferent from 'Not assigned'
df = df[df.Borough != 'Not assigned']

#2)If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

#3)Grouping per postcode and bringing neighborhoods and boroughs to the same line  
df=df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head(15)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [53]:
df.shape

(103, 3)